In [ ]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import z_NB
import numpy as np

In [ ]:
with open('npy/selection.npy', 'rb') as f:
    selection = pickle.load(f)
selection.keys()

In [ ]:
# Make DataFrame
df = pd.DataFrame(data=selection)

# Drop some columns
for col in ['src', 'x_im', 'y_im']:
    df = df.drop(col, axis=1)

# Convert tile_id into tile names
tile_dict = {
    2241: 'AEGIS001',
    2243: 'AEGIS002',
    2406: 'AEGIS003',
    2470: 'AEGIS004',
    2520: 'J-NEP'
}

def tile_id_to_name(id):
    return tile_dict[int(id)]

def dec_to_sex(alpha):
    alpha = alpha * 60 * 60
    m, s = divmod(alpha, 60)
    d, m = divmod(m, 60)
    return f'{d:0.0f}º{m:0.0f}``{s:0.1f}\''

def redshift(nb):
    return f'{z_NB(nb)[0]:0.2f}'

def substitute_nans_zspec(what):
    if np.isnan(what):
        return '-'
    else:
        return f'{np.abs(what):0.2f}'

def substitute_nans_spcl(what):
    if what == 'nan':
        return '-'
    else:
        return what

df['tile_id'] = df['tile_id'].astype(int).apply(tile_id_to_name)
df['RA'] = df['RA'].apply(dec_to_sex)
df['DEC'] = df['DEC'].apply(dec_to_sex)
df['nb_sel'] = df['nb_sel'].apply(redshift)
df['SDSS_spCl'] = df['SDSS_spCl'].astype(str).apply(substitute_nans_spcl)
df['SDSS_zspec'] = df['SDSS_zspec'].apply(substitute_nans_zspec)

ew_np = df['EW_lya'].to_numpy()
ew_err_np = df['EW_lya_err'].to_numpy()
ew_str = np.zeros(ew_np.shape).astype(str)
for i, (ew, err) in enumerate(zip(ew_np, ew_err_np)):
    ew_str[i] = f'{ew:0.0f}$\pm${err:0.0f}'
df['EW_lya'] = ew_str
df = df.drop('EW_lya_err', axis=1)

L_np = df['L_lya'].to_numpy()
L_err_np = df['L_lya_err'].to_numpy()
L_str = np.zeros(L_np.shape).astype(str)
for i, (L, err) in enumerate(zip(L_np, L_err_np)):
    L_str[i] = f'{L:0.2f}$\pm${err:0.2f}'
df['L_lya'] = L_str
df = df.drop('L_lya_err', axis=1)

# Sort by tile_id
df = df.sort_values('tile_id')
df.head()

In [ ]:
df.to_csv('csv/selection_table.csv')